In [145]:
from operator import add
from pyspark.sql import SparkSession
import sys
import pyspark.sql.functions
import pandas as pd

In [146]:
spark = SparkSession.builder.appName("sexAgeRatio").getOrCreate()


In [147]:
df1 = spark.read.options(header='True', inferSchema='False', delimiter=',')\
.csv("PycharmProjects/FBD-Final/src/user_log_format1.csv")

In [148]:
df1.show()

+-------+-------+------+---------+--------+----------+-----------+
|user_id|item_id|cat_id|seller_id|brand_id|time_stamp|action_type|
+-------+-------+------+---------+--------+----------+-----------+
| 328862| 323294|   833|     2882|    2661|      0829|          0|
| 328862| 844400|  1271|     2882|    2661|      0829|          0|
| 328862| 575153|  1271|     2882|    2661|      0829|          0|
| 328862| 996875|  1271|     2882|    2661|      0829|          0|
| 328862|1086186|  1271|     1253|    1049|      0829|          0|
| 328862| 623866|  1271|     2882|    2661|      0829|          0|
| 328862| 542871|  1467|     2882|    2661|      0829|          0|
| 328862| 536347|  1095|      883|    1647|      0829|          0|
| 328862| 364513|  1271|     2882|    2661|      0829|          0|
| 328862| 575153|  1271|     2882|    2661|      0829|          0|
| 328862| 239288|   602|      420|    4953|      0801|          0|
| 328862| 950862|   962|     4605|    7622|      0801|        

In [149]:
df1=df1.filter(df1.action_type=='2').select('user_id').distinct()

In [150]:
df1.show()

+-------+
|user_id|
+-------+
| 100263|
| 148631|
|  88140|
| 225241|
| 271564|
| 237724|
|  81824|
| 179022|
| 363694|
| 319755|
| 223506|
| 254821|
|  59072|
| 361186|
| 419208|
|  51550|
| 155658|
| 323208|
|  91959|
| 151010|
+-------+
only showing top 20 rows



In [158]:
df2 = spark.read.options(header='True', inferSchema='False', delimiter=',')\
.csv("PycharmProjects/FBD-Final/src/user_info_format1.csv")

In [159]:
df2.show()

+-------+---------+------+
|user_id|age_range|gender|
+-------+---------+------+
| 376517|        6|     1|
| 234512|        5|     0|
| 344532|        5|     0|
| 186135|        5|     0|
|  30230|        5|     0|
| 272389|        6|     1|
| 281071|        4|     0|
| 139859|        7|     0|
| 198411|        5|     1|
|  67037|        4|     1|
| 149002|        5|     2|
|   7468|        4|     0|
|  94292|        4|     0|
| 347414|        6|     1|
| 191719|        4|     0|
| 391524|        5|     1|
| 153790|        6|     0|
| 349112|        3|     1|
| 344766|        6|     0|
|  81816|        5|     0|
+-------+---------+------+
only showing top 20 rows



In [161]:
df2=df2.filter(df2.age_range<"9").filter(df2.age_range>"0").filter(df2.gender!="2").filter(df2.gender!="")\
.dropDuplicates(subset=[c for c in df.columns if c in ["user_id"]])

In [162]:
df2.show()

+-------+---------+------+
|user_id|age_range|gender|
+-------+---------+------+
| 100010|        5|     1|
| 100140|        6|     1|
| 100227|        4|     0|
| 100263|        5|     0|
| 100320|        5|     0|
| 100553|        4|     1|
| 100704|        3|     0|
| 100768|        4|     1|
|  10096|        3|     1|
| 100964|        5|     1|
| 101021|        3|     1|
| 101122|        4|     1|
| 101205|        3|     0|
| 101261|        6|     0|
| 101272|        2|     0|
| 102113|        5|     0|
| 102536|        3|     0|
| 102539|        3|     0|
| 102684|        6|     0|
| 102745|        7|     1|
+-------+---------+------+
only showing top 20 rows



In [163]:
df3=df1.join(df2, "user_id", "left").select("user_id", "age_range", "gender")

In [164]:
df3.show()

+-------+---------+------+
|user_id|age_range|gender|
+-------+---------+------+
| 100263|        5|     0|
| 148631|        5|     1|
|  88140|        5|     1|
| 225241|     null|  null|
| 271564|     null|  null|
| 237724|        5|     1|
|  81824|        4|     1|
| 179022|        4|     1|
| 363694|        4|     0|
| 319755|        5|     0|
| 223506|        4|     0|
| 254821|     null|  null|
|  59072|        3|     1|
| 361186|        4|     0|
| 419208|        6|     1|
|  51550|        6|     1|
| 155658|        3|     0|
| 323208|        4|     0|
|  91959|     null|  null|
| 151010|        5|     1|
+-------+---------+------+
only showing top 20 rows



In [165]:
resSex = df3.groupby("gender").count()

In [166]:
resSex.show()

+------+------+
|gender| count|
+------+------+
|     0|218825|
|  null|106330|
|     1| 99015|
+------+------+



In [167]:
resAge = df3.groupby("age_range").count()

In [168]:
resAge.show()

+---------+------+
|age_range| count|
+---------+------+
|        7|  6849|
|        3|107826|
|        8|  1210|
|     null|106330|
|        5| 39285|
|        6| 34905|
|        1|    24|
|        4| 75740|
|        2| 52001|
+---------+------+



In [141]:
#resAge.write.csv("PycharmProjects/FBD-Final/output/test.csv" ) 似乎不行，每一行都是一个文件

In [144]:
resAge.toPandas().to_csv('PycharmProjects/FBD-Final/output/任务三/ageRatioSQL.csv')
resSex.toPandas().to_csv('PycharmProjects/FBD-Final/output/任务三/sexRatioSQL.csv')